# HW2 - Amit Parizat & Yoel Melul

## Theory Questions

### Q1
To evaluate how well our model performs at T1D classification, we need to have evaluation metrics that measures of its performances/accuracy. Which evaluation metric is more important to us: model accuracy or model performance? Give a simple example that illustrates your claim.

In our model, the more important evaluation metric is model performance. Like we saw in lecture "*C08 - Practical consideration on training a model II*" in slide 8, accuracy could be high even when the classifier misclassifies most of 'Yes' labeled examples. For example, when 99% percent of the data is labeled 'No' and only 1% is labeled 'Yes' (could be our case since most of the population doesn't have diabetes), even when the classifier labeles 'No' all the examples we obtain high accuracy. In this case, performance statistics such as sensitivity, PPV and F1 would be more informative.

### Q2
T1D is often associated with other comorbidities such as a heart attack. You are asked to design a ML algorithm to predict which patients are going to suffer a heart attack. Relevant patient features for the algorithm may include blood pressure (BP), body-mass index (BMI), age (A), level of physical activity (P), and income (I). You should choose between two classifiers: the first uses only BP and BMI features and the other one uses all of the features available to you. Explain the pros and cons of each choice.

When using only two features we create a simpler model that requires less computetional effort to be made and less memory space to accommodate weights or any other information about the model. However, when not taking into consideration all of the data provided, we may miss an important information that may have high indecativity on heart attacks. We may also miss trasformations of the neglected features that could yield valueable new features.

### Q3
A histologist wants to use machine learning to tell the difference between pancreas biopsies that show signs of T1D and those that do not. She has already come up with dozens of measurements to take, such as color, size, uniformity and cell-count, but she isn’t sure which model to use. The biopsies are really similar, and it is difficult to distinguish them from the human eye, or by just looking at the features. Which of the following is better: logistic regression, linear SVM or nonlinear SVM? Explain your answer.

### Q4
What are the differences between LR and linear SVM and what is the difference in the effect/concept of their hyper-parameters tuning?

## Coding Assignment

### Part 1
Load the data. Explain any preprocessing.

### Part 2
Perform a test-train split of 20% test.

### Part 3
Provide a detailed visualization and exploration of the data.

### Part 4
Encode all your data as one hot vectors.

### Part 5
Choose, build and optimize Machine Learning Models.

### Part 6
Feature Selection

### Part 7
Data Separability Visualization